# Routing Analysis Notebook
This notebook contains the queries for the analysis of Iris measurements. The goal is to find how much of the routing information can Iris extract by it's measurements.

__note:__
Here the measurement `007046a9_518e_46cb_8e70_e598b8bce831` is selected arbitrarily. The metadata should also be considered.

In [2]:
%pip install pandas requests python-dotenv

import util as util
import pandas as pd 
import requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
client = util.IrisClickhouseClient()
choosen_measurement_uuid = "007046a9_518e_46cb_8e70_e598b8bce831" # get info from PostgreSQL

In [4]:
# Some utility functions

def union_of_all(table_names: list[str], database: str="iris", limit_each: int=0) -> str:
    if isinstance(table_names, pd.DataFrame):
        table_names = list(table_names['name'])

    tables = []
    limit_str = ""
    if limit_each != 0:
          limit_str = f" LIMIT {limit_each}"

    for i, table_name in enumerate(table_names):
        tables.append(f"SELECT * FROM {database}.{table_name}{limit_str}")
    return " UNION ALL ".join(tables)
    

In [5]:
q = f'''
    SELECT name 
    FROM system.tables 
    WHERE name LIKE 'cleaned_results__{choosen_measurement_uuid}%';
'''
selected_tables_df = client.query_df(q)
set(selected_tables_df['name'])

{'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__51fb56f4_5217_4108_a212_2d05806b16dc',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__5bdab212_3d82_4eca_8c5a_39e1c8ded842',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__70ce196e_9cb7_4467_b1a8_c40400f6f5df',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__91b015da_729c_40cd_b051_d6439ce5c00d',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__a05d071c_159d_448e_a349_95431bf96a51',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__bc17c7f9_9d02_48de_b766_8c927c8c10fd',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__ccdf7970_8877_42cc_960d_41d0d0d0c302',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__f39c7a18_5b00_4e57_9694_f3db8198b72a'}

In [ ]:
q = f'''
WITH 
total AS (
    SELECT * 
    FROM ({union_of_all(selected_tables_df)})
),
s2r AS (
    SELECT probe_src_addr AS source, reply_src_addr AS destination
    FROM total
),
s2d AS (
    SELECT probe_src_addr AS source, probe_dst_addr AS destination
    FROM total
),
r2d AS (
    SELECT reply_src_addr AS source, probe_dst_addr AS destination
    FROM total
),
unique_pairs AS (
    SELECT  
        DISTINCT source, destination
    FROM (
        SELECT * FROM s2r -- Source to Router
        UNION ALL 
        SELECT * FROM s2d -- Source to Destination
        UNION ALL 
        SELECT * FROM r2d -- Rotuer to Destination
    )
)
SELECT COUNT(*) FROM unique_pairs;
'''
num_distinct_pairs_df = client.query_df(q)
num_distinct_pairs_df

Error while query:500 Server Error: Internal Server Error for url: https://chproxy.iris.dioptra.io/?enable_http_compression=false&default_format=CSVWithNames&output_format_json_quote_64bit_integer=&database=iris&query=%0AWITH+%0Atotal+AS+%28%0A++++SELECT+%2A+%0A++++FROM+%28SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__51fb56f4_5217_4108_a212_2d05806b16dc+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__5bdab212_3d82_4eca_8c5a_39e1c8ded842+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__70ce196e_9cb7_4467_b1a8_c40400f6f5df+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__91b015da_729c_40cd_b051_d6439ce5c00

In [6]:
# q = f'''
#     DESCRIBE TABLE cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604;
# '''
# columns_described_df = client.query_df(q)
# columns_described_df

In [6]:
q = f'''
    SELECT * FROM cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604 LIMIT 10;
'''
first_10_rows_df = client.query_df(q)
first_10_rows_df

,capture_timestamp,probe_protocol,probe_src_addr,probe_dst_addr,probe_src_port,probe_dst_port,probe_ttl,quoted_ttl,reply_src_addr,reply_protocol,...,rtt,round,probe_dst_prefix,reply_src_prefix,private_probe_dst_prefix,private_reply_src_addr,destination_host_reply,destination_prefix_reply,valid_probe_protocol,time_exceeded_reply
0,2024-05-22 02:26:07,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,4,1,::ffff:203.190.250.176,1,...,563,1,::ffff:1.0.213.0,::ffff:203.190.250.0,0,0,0,0,1,1
1,2024-05-22 02:26:07,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,5,1,::ffff:203.190.251.159,1,...,560,1,::ffff:1.0.213.0,::ffff:203.190.251.0,0,0,0,0,1,1
2,2024-05-22 02:26:07,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,6,1,::ffff:203.113.59.131,1,...,646,1,::ffff:1.0.213.0,::ffff:203.113.59.0,0,0,0,0,1,1
3,2024-05-22 02:26:07,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,7,2,::ffff:125.24.217.162,1,...,636,1,::ffff:1.0.213.0,::ffff:125.24.217.0,0,0,0,0,1,1
4,2024-05-22 02:26:07,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,8,1,::ffff:203.113.44.217,1,...,644,1,::ffff:1.0.213.0,::ffff:203.113.44.0,0,0,0,0,1,1
5,2024-05-22 02:26:07,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,10,0,::ffff:1.0.213.11,1,...,681,1,::ffff:1.0.213.0,::ffff:1.0.213.0,0,0,1,1,1,0
6,2024-05-22 02:29:31,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,11,0,::ffff:1.0.213.11,1,...,679,1,::ffff:1.0.213.0,::ffff:1.0.213.0,0,0,1,1,1,0
7,2024-05-22 02:29:31,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,12,0,::ffff:1.0.213.11,1,...,681,1,::ffff:1.0.213.0,::ffff:1.0.213.0,0,0,1,1,1,0
8,2024-05-22 02:29:31,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,13,0,::ffff:1.0.213.11,1,...,678,1,::ffff:1.0.213.0,::ffff:1.0.213.0,0,0,1,1,1,0
9,2024-05-22 02:29:31,1,::ffff:10.0.0.2,::ffff:1.0.213.11,24000,0,14,0,::ffff:1.0.213.11,1,...,680,1,::ffff:1.0.213.0,::ffff:1.0.213.0,0,0,1,1,1,0


See the progress report on [Obsidian](obsidian://open?vault=Doctoral%20Vault&file=P%202024-11-26%20Iris%20Data%20Analysis).

In an ideal situation there are 3 nodes, _source_, _router_, _destination_ where they have the following addresses:
- _source_: `probe_src_addr`, `reply_dst_addr`
- _router_: `repy_src_addr`
- _destination_: `probe_dst_addr`

We can deduce 3 pieces of routing information. _source-router_, _router-destination_, _source-destination_. So to have them we will crate 3 queries and merge them to see the counts.


In [7]:
q = f'''
    SELECT COUNT(*) FROM ({union_of_all(selected_tables_df)});
'''
number_of_rows_total_df = client.query_df(q)
number_of_rows_total = int(number_of_rows_total_df['count()'].iloc[0])
number_of_rows_total

758301083

In [8]:
q = f'''
    WITH results as ({union_of_all(selected_tables_df)})
    SELECT COUNT(*), IPv6ToIPv4(results.reply_src_addr) AS reply_src_addr_ipv4
    FROM ({union_of_all(selected_tables_df)})
    WHERE isIPv4(IPv6ToIPv4(reply_src_addr_ipv4));
'''
client.query_df(q)

Error while query:404 Client Error: Not Found for url: https://chproxy.iris.dioptra.io/?enable_http_compression=false&default_format=CSVWithNames&output_format_json_quote_64bit_integer=&database=iris&query=%0A++++WITH+results+as+%28SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__51fb56f4_5217_4108_a212_2d05806b16dc+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__5bdab212_3d82_4eca_8c5a_39e1c8ded842+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__70ce196e_9cb7_4467_b1a8_c40400f6f5df+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__91b015da_729c_40cd_b051_d6439ce5c00d+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_r

# Router to Destination Routes
> 233341495

In [9]:
q = f'''
    WITH router_to_destination AS ({union_of_all(selected_tables_df)})
    SELECT COUNT(DISTINCT (router_to_destination.reply_src_addr, router_to_destination.probe_dst_addr))
    FROM router_to_destination 
'''
num_router_to_destination_df = client.query_df(q)
num_router_to_destination_df

,"uniqExact(tuple(reply_src_addr, probe_dst_addr))"
0,233341495


# Source To Router
> 10828358

In [10]:
q = f'''
    WITH router_to_destination AS ({union_of_all(selected_tables_df)})
    SELECT COUNT(DISTINCT (router_to_destination.probe_src_addr, router_to_destination.reply_src_addr))
    FROM router_to_destination 
'''
num_source_to_router_df = client.query_df(q)
num_source_to_router_df

,"uniqExact(tuple(probe_src_addr, reply_src_addr))"
0,10828358


# Source To Destination
> 10828358

NameError: name 'union_of_all' is not defined

In [ ]:
q = f'''
     WITH r2d as (
        WITH router_to_destination AS ({union_of_all(selected_tables_df)})
        SELECT DISTINCT (router_to_destination.probe_src_addr, router_to_destination.reply_src_addr)
        FROM router_to_destination 
     )   


    WITH RECURSIVE Reachability AS (
    -- Base case: Direct connections from the edges table
    SELECT 
        source, 
        destination 
    FROM 
        edges
    UNION ALL
    -- Recursive step: Add paths by extending existing paths
    SELECT 
        r.source, 
        e.destination
    FROM 
        Reachability r
    JOIN 
        edges e
    ON 
        r.destination = e.source
)
-- Calculate the sum of all unique pairs in the reachability matrix
SELECT 
    COUNT(*) AS reachability_sum
FROM 
    Reachability;
    SELECT COUNT(DISTINCT (router_to_destination.probe_src_addr, router_to_destination.probe_dst_addr))
    FROM router_to_destination 
'''
num_source_to_destination_df = client.query_df(q)
num_source_to_router_df

Error while query:400 Client Error: Bad Request for url: https://chproxy.iris.dioptra.io/?enable_http_compression=false&default_format=CSVWithNames&output_format_json_quote_64bit_integer=&database=iris&query=%0A++++WITH+router_to_destination+AS+%28SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__51fb56f4_5217_4108_a212_2d05806b16dc+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__5bdab212_3d82_4eca_8c5a_39e1c8ded842+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__70ce196e_9cb7_4467_b1a8_c40400f6f5df+UNION+ALL+SELECT+%2A+FROM+iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__91b015da_729c_40cd_b051_d6439ce5c00d+UNION+ALL+SELECT+%2A+FRO

,"uniqExact(tuple(probe_src_addr, reply_src_addr))"
0,10828358


In [12]:
q = '''
    WITH RECURSIVE Reachability AS (
        -- Base case: Direct connections from the edges table
        SELECT 
            source, 
            destination 
        FROM 
            edges
        UNION ALL
        -- Recursive step: Add paths by extending existing paths
        SELECT 
            r.source, 
            e.destination
        FROM 
            Reachability r
        JOIN 
            edges e
        ON 
            r.destination = e.source
    )
    -- Calculate the sum of all unique pairs in the reachability matrix
    SELECT 
        COUNT(*) AS reachability_sum
    FROM 
        Reachability;
'''

In [13]:
# IPv6ToIPv4(ipv6_column) AS ipv4_address

In [14]:
def create_values(records: pd.DataFrame, table: str):
    header = f"""INSERT INTO iris.{table} VALUES """
    values = []
    for row in records.itertuples():
        row_as_list = tuple([x for x in row][1:])
        values.append(str(row_as_list))
    return header + (",\n".join(values))
        

# v = create_values(client.query_df("SELECT * FROM cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604 LIMIT 100000;"), "cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604")
# print(v)

In [15]:
q = f'''
    SELECT name 
    FROM system.tables 
    WHERE name LIKE 'cleaned_%{choosen_measurement_uuid}%';
'''
selected_tables_df = client.query_df(q)
set(selected_tables_df['name'])

{'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__51fb56f4_5217_4108_a212_2d05806b16dc',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__5bdab212_3d82_4eca_8c5a_39e1c8ded842',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__70ce196e_9cb7_4467_b1a8_c40400f6f5df',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__91b015da_729c_40cd_b051_d6439ce5c00d',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__a05d071c_159d_448e_a349_95431bf96a51',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__bc17c7f9_9d02_48de_b766_8c927c8c10fd',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__ccdf7970_8877_42cc_960d_41d0d0d0c302',
 'cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__f39c7a18_5b00_4e57_9694_f3db8198b72a',
 'cleaned_prefixes__007046a9_518e_46cb_8e70_e598b8bce831__40

In [16]:
q = f'''
    SELECT * 
    FROM cleaned_prefixes__007046a9_518e_46cb_8e70_e598b8bce831__a05d071c_159d_448e_a349_95431bf96a51
    WHERE has_amplification > 0
    LIMIT 10 
'''
return_df = client.query_df(q)
return_df

,probe_protocol,probe_src_addr,probe_dst_prefix,has_amplification,has_loops
0,1,::ffff:10.0.4.2,::ffff:5.181.159.0,1,1
1,1,::ffff:10.0.4.2,::ffff:23.232.53.0,1,1
2,1,::ffff:10.0.4.2,::ffff:23.232.61.0,1,1
3,1,::ffff:10.0.4.2,::ffff:23.255.1.0,1,0
4,1,::ffff:10.0.4.2,::ffff:27.71.230.0,1,1
5,1,::ffff:10.0.4.2,::ffff:37.203.190.0,1,1
6,1,::ffff:10.0.4.2,::ffff:41.141.51.0,1,1
7,1,::ffff:10.0.4.2,::ffff:41.143.87.0,1,1
8,1,::ffff:10.0.4.2,::ffff:41.174.165.0,1,1
9,1,::ffff:10.0.4.2,::ffff:41.250.93.0,1,1


In [17]:
q = f'''
    SELECT * FROM cleaned_probes__007046a9_518e_46cb_8e70_e598b8bce831__a05d071c_159d_448e_a349_95431bf96a51
      limit 10;
'''
return_df = client.query_df(q)
return_df

,probe_protocol,probe_dst_prefix,probe_ttl,cumulative_probes,round
0,1,::ffff:0.0.42.0,4,6,1
1,1,::ffff:0.0.42.0,5,6,1
2,1,::ffff:0.0.42.0,6,6,1
3,1,::ffff:0.0.42.0,7,6,1
4,1,::ffff:0.0.42.0,8,6,1
5,1,::ffff:0.0.42.0,9,6,1
6,1,::ffff:0.0.42.0,10,6,1
7,1,::ffff:0.0.52.0,4,6,1
8,1,::ffff:0.0.52.0,5,6,1
9,1,::ffff:0.0.52.0,6,6,1


In [18]:
q = f'''
    SELECT * 
    FROM cleaned_links__007046a9_518e_46cb_8e70_e598b8bce831__a05d071c_159d_448e_a349_95431bf96a51
    LIMIT 10 
'''
return_df = client.query_df(q)
return_df

,probe_protocol,probe_src_addr,probe_dst_prefix,probe_dst_addr,probe_src_port,probe_dst_port,near_round,far_round,near_ttl,far_ttl,near_addr,far_addr
0,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.13,24000,0,1,1,4,5,::ffff:210.166.33.163,::ffff:210.166.42.230
1,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.13,24000,0,1,1,5,6,::ffff:210.166.42.230,::ffff:219.118.242.14
2,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.71,24000,0,1,1,4,5,::ffff:210.166.33.163,::ffff:210.166.42.230
3,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.71,24000,0,1,1,5,6,::ffff:210.166.42.230,::ffff:219.118.242.14
4,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.80,24000,0,2,2,4,5,::ffff:210.166.33.163,::ffff:210.166.42.230
5,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.80,24000,0,2,2,5,6,::ffff:210.166.42.230,::ffff:219.118.242.14
6,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.91,24000,0,1,1,4,5,::ffff:210.166.33.179,::ffff:210.166.42.230
7,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.91,24000,0,1,1,5,6,::ffff:210.166.42.230,::ffff:219.118.242.14
8,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.92,24000,0,1,1,4,5,::ffff:210.166.33.163,::ffff:210.166.42.230
9,1,::ffff:10.0.4.2,::ffff:1.0.92.0,::ffff:1.0.92.92,24000,0,1,1,5,6,::ffff:210.166.42.230,::ffff:219.118.242.14
